<a href="https://colab.research.google.com/github/Aipakazuma/meetup037/blob/master/tensorboard%E3%81%AE%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E7%A2%BA%E8%AA%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorboardのパラメータ確認

* 画像とかhistogramとか出したい

In [0]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.utils import np_utils


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# tensorboardを使う準備

`localtunnel` というコマンドを使ってトンネルを作る

In [0]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/client
+ localtunnel@1.9.1
updated 1 package in 1.073s


In [0]:
!rm -rf ./logs/

In [0]:
log_dir = './logs/'

In [0]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(log_dir)
)
get_ipython().system_raw('lt --port 6006 > url.txt 2>&1 &')

In [0]:
!cat url.txt

your url is: https://wet-dodo-95.localtunnel.me


# データダウンロード

In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# 前処理

In [0]:
# 画像を0-1の範囲で正規化
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# 正解ラベルをOne-Hot表現に変換
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# モデル構築

In [0]:
model = Sequential()
model.add(Flatten(input_shape=(32, 32, 3)))
model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                98336     
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
_________________________________________________________________
activation_3 (Activation)    (None, 10)                0         
Total para

# callbackでtensorboardを読み出す

In [0]:
from keras.callbacks import TensorBoard

In [0]:
batch_size = 128

In [0]:
tensorboard = TensorBoard(log_dir=log_dir,
                          histogram_freq=True,  # histogramの出力. 各レイヤーの値の分布を表示(3次元)
                          batch_size=batch_size,  # batch_sizeの指定
                          write_grads=True,  # 各レイヤーの値の分布の表示(2次元)
                          write_images=True,  # 各レイヤーの重み、バイアスを画像で表示
                          embeddings_freq=False)
callbacks = [tensorboard]

# モデル学習

In [21]:
epochs = 50
try:
    history = model.fit(X_train, y_train, epochs=epochs, callbacks=callbacks, validation_data=(X_test, y_test))
except KeyboardInterrupt:
    tensorboard.writer.close()

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 13s 252us/step - loss: 1.9348 - acc: 0.2933 - val_loss: 1.8267 - val_acc: 0.3358
Epoch 2/50
50000/50000 [==============================] - 12s 249us/step - loss: 1.8164 - acc: 0.3434 - val_loss: 1.7790 - val_acc: 0.3607
Epoch 3/50
50000/50000 [==============================] - 12s 248us/step - loss: 1.7973 - acc: 0.3510 - val_loss: 1.7963 - val_acc: 0.3526
Epoch 4/50
50000/50000 [==============================] - 13s 251us/step - loss: 1.7744 - acc: 0.3609 - val_loss: 1.7577 - val_acc: 0.3644
Epoch 5/50
50000/50000 [==============================] - 12s 249us/step - loss: 1.7603 - acc: 0.3655 - val_loss: 1.7576 - val_acc: 0.3693
Epoch 6/50
50000/50000 [==============================] - 12s 245us/step - loss: 1.7527 - acc: 0.3690 - val_loss: 1.7570 - val_acc: 0.3671
Epoch 7/50
50000/50000 [==============================] - 12s 246us/step - loss: 1.7470 - acc: 0.3688 - val_loss: 1.

動作させている間はTensorboardに動的に書き込みされる

In [27]:
history.history.keys()

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])